In [1]:
import itertools
import os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import re
from sklearn.metrics import accuracy_score, classification_report

import tensorflow

from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.utils import to_categorical
import statsmodels.api as sm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df1_tabloids = pd.read_csv("Dimensions_texts_Tabloids.csv")
df2_tabloids = pd.read_csv("Statistics_texts_Tabloids.csv")

In [3]:
df1_broadsheets = pd.read_csv("Dimensions_texts_Broadsheets.csv")
df2_broadsheets = pd.read_csv("Statistics_texts_Broadsheets.csv")

In [4]:
df1_tabloids.columns

Index(['Filename', 'Dimension1', 'Dimension2', 'Dimension3', 'Dimension4',
       'Dimension5', 'Dimension6', 'Closest Text Type'],
      dtype='object')

In [5]:
df2_broadsheets.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', '#', '$', '''', ':', 'AMP', 'ANDC',
       'CAUS', 'CC', 'CD', 'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DPAR',
       'DT', 'DWNT', 'EMPH', 'EX', 'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR',
       'JJ', 'NEMD', 'NN', 'NOMZ', 'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE',
       'POMD', 'POS', 'PRED', 'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP',
       'SPP2', 'SYM', 'SYNE', 'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3',
       'TSUB', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS',
       'XX0', '[BEMA]', '[BYPA]', '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]',
       '[PRIV]', '[PROD]', '[PUBV]', '[SERE]', '[SMP]', '[SPAU]', '[SPIN]',
       '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHQU]', '[WHSUB]',
       '[WZPAST]', '[WZPRES]', '``', 'Unnamed: 90'],
      dtype='object')

In [6]:
df2_broadsheets = df2_broadsheets.drop(columns = ['Unnamed: 90'])
df2_tabloids = df2_tabloids.drop(columns = ['Unnamed: 89'])

In [7]:
y_tabloids = df1_tabloids['Closest Text Type']
y_broadsheets = df1_broadsheets['Closest Text Type']

In [8]:
df1_broadsheets

,Filename,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6,Closest Text Type
0,text_0,-3.75,4.07,7.59,-2.18,-1.74,-0.41,General narrative exposition
1,text_100,-18.97,-5.83,0.30,-1.45,-1.95,0.84,Learned exposition
2,text_101,-2.72,-0.20,1.91,-3.49,1.70,-1.05,General narrative exposition
3,text_102,-18.69,-5.26,5.77,-4.90,-0.96,-1.24,Learned exposition
4,text_103,-15.85,-3.88,1.94,6.77,5.34,-2.12,Scientific exposition
...,...,...,...,...,...,...,...,...
132,text_97,-11.01,-2.96,4.25,1.60,-0.63,0.48,General narrative exposition
133,text_98,-13.76,-2.60,0.69,-3.81,2.51,0.00,General narrative exposition
134,text_99,-4.28,4.06,6.07,-1.87,3.20,0.73,General narrative exposition
135,text_9,-14.07,-1.63,6.47,6.77,-1.32,-0.39,General narrative exposition


In [9]:
df2_tabloids['Closest Text Type'] = df1_tabloids['Closest Text Type']
df2_broadsheets['Closest Text Type'] = df1_broadsheets['Closest Text Type']

In [10]:
bads = []

for l in list(df2_tabloids.columns):
    if re.findall('[a-zA-Z]', l) == []:
        bads.append(l)

for l in list(df2_broadsheets.columns):
    if re.findall('[a-zA-Z]', l) == []:
        bads.append(l)

In [11]:
for l in list(df2_tabloids.columns):
    if l not in df2_broadsheets:
        print(l)

df2_tabloids = df2_tabloids.drop(["-RRB-", '#', '$', "''", ':'], axis = 1)

-RRB-


In [12]:
for l in list(df2_broadsheets.columns):
    if l not in df2_tabloids:
        print(l)

df2_broadsheets = df2_broadsheets.drop(["DPAR", "``", '#', '$', "''", ':'], axis = 1)

#
$
''
:
DPAR
``


In [13]:
df2_tabloids.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD',
       'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DT', 'DWNT', 'EMPH', 'EX',
       'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'NEMD', 'NN', 'NOMZ',
       'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED',
       'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE',
       'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD',
       'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]',
       '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRIV]', '[PROD]', '[PUBV]',
       '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[SUAV]', '[THATD]', '[WHCL]',
       '[WHOBJ]', '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]',
       'Closest Text Type'],
      dtype='object')

In [14]:
df2_broadsheets.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD',
       'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DT', 'DWNT', 'EMPH', 'EX',
       'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'NEMD', 'NN', 'NOMZ',
       'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED',
       'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE',
       'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD',
       'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]',
       '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRIV]', '[PROD]', '[PUBV]',
       '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[SUAV]', '[THATD]', '[WHCL]',
       '[WHOBJ]', '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]',
       'Closest Text Type'],
      dtype='object')

In [15]:
df2_tabloids

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHQU],[WHSUB],[WZPAST],[WZPRES],Closest Text Type
0,text_0,1040.00,4.15,169.00,0.29,0.10,0.00,2.02,1.25,0.00,...,0.00,0.10,0.87,0.00,0.19,0.0,0.77,0.19,0.00,General narrative exposition
1,text_100,559.00,4.15,203.00,0.36,0.00,0.00,2.15,0.00,0.00,...,0.00,0.36,1.07,0.36,0.18,0.0,0.00,0.00,0.00,Involved persuasion
2,text_101,476.00,3.91,174.00,0.00,0.00,0.00,2.73,1.26,0.00,...,0.00,0.21,0.63,0.42,0.21,0.0,0.21,0.21,0.21,General narrative exposition
3,text_102,315.00,4.16,165.00,0.32,0.00,0.00,1.59,1.27,0.00,...,0.00,0.00,2.22,0.00,0.00,0.0,0.63,0.00,0.00,General narrative exposition
4,text_103,443.00,4.47,181.00,0.23,0.00,0.23,1.35,1.13,0.00,...,0.00,0.23,1.13,0.00,0.23,0.0,0.00,0.00,0.23,General narrative exposition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,text_97,1090.00,4.36,194.00,0.00,0.09,0.00,2.39,1.01,0.00,...,0.09,0.64,1.56,0.09,0.09,0.0,0.28,0.00,0.00,General narrative exposition
131,text_98,428.00,4.44,211.00,0.00,0.00,0.00,1.87,0.93,0.00,...,0.00,0.00,0.93,0.00,0.00,0.0,0.93,0.00,0.00,General narrative exposition
132,text_99,508.00,4.84,174.00,0.00,0.00,0.00,1.38,1.77,0.00,...,0.00,0.39,0.79,0.20,0.00,0.0,0.39,0.00,0.00,Learned exposition
133,text_9,329.00,4.56,165.00,0.00,0.00,0.00,2.13,0.30,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,Learned exposition


In [16]:
df1_broadsheets = df1_broadsheets.drop(['Filename'],axis=1)
df1_tabloids = df1_tabloids.drop(['Filename'],axis=1)
df2_broadsheets = df2_broadsheets.drop(['Filename'],axis=1)
df2_tabloids = df2_tabloids.drop(['Filename'],axis=1)

In [17]:
df1_broadsheets['Closest Text Type'].unique()

array(['General narrative exposition', 'Learned exposition',
       'Scientific exposition', 'Involved persuasion'], dtype=object)

In [18]:
df1_tabloids['Closest Text Type'].unique()

array(['General narrative exposition', 'Involved persuasion',
       'Learned exposition', 'Scientific exposition'], dtype=object)

In [19]:
df1_br_types = []

for d in df1_broadsheets['Closest Text Type']:
    if d == 'General narrative exposition':
        df1_br_types.append(0)
    elif d == 'Imaginative narrative':
        df1_br_types.append(1)
    elif d == 'Informational interaction':
        df1_br_types.append(2)
    elif d == 'Involved persuasion':
        df1_br_types.append(3)
    elif d == 'Learned exposition':
        df1_br_types.append(4)
    elif d == 'Scientific exposition':
        df1_br_types.append(5)
    else:
        df1_br_types.append(6)

In [20]:
df1_tabl_types = []

for d in df1_tabloids['Closest Text Type']:
    if d == 'General narrative exposition':
        df1_tabl_types.append(0)
    elif d == 'Imaginative narrative':
        df1_tabl_types.append(1)
    elif d == 'Informational interaction':
        df1_tabl_types.append(2)
    elif d == 'Involved persuasion':
        df1_tabl_types.append(3)
    elif d == 'Learned exposition':
        df1_tabl_types.append(4)
    elif d == 'Scientific exposition':
        df1_tabl_types.append(5)
    else:
        df1_tabl_types.append(6)

In [21]:
df2_br_types = []

for d in df2_broadsheets['Closest Text Type']:
    if d == 'General narrative exposition':
        df2_br_types.append(0)
    elif d == 'Imaginative narrative':
        df2_br_types.append(1)
    elif d == 'Informational interaction':
        df2_br_types.append(2)
    elif d == 'Involved persuasion':
        df2_br_types.append(3)
    elif d == 'Learned exposition':
        df2_br_types.append(4)
    elif d == 'Scientific exposition':
        df2_br_types.append(5)
    else:
        df2_br_types.append(6)

In [22]:
df2_tabl_types = []

for d in df2_tabloids['Closest Text Type']:
    if d == 'General narrative exposition':
        df2_tabl_types.append(0)
    elif d == 'Imaginative narrative':
        df2_tabl_types.append(1)
    elif d == 'Informational interaction':
        df2_tabl_types.append(2)
    elif d == 'Involved persuasion':
        df2_tabl_types.append(3)
    elif d == 'Learned exposition':
        df2_tabl_types.append(4)
    elif d == 'Scientific exposition':
        df2_tabl_types.append(5)
    else:
        df2_tabl_types.append(6)

In [23]:
df1_tabloids['Closest_Text_Type'] = df1_tabl_types
df2_tabloids['Closest_Text_Type'] = df2_tabl_types
df1_broadsheets['Closest_Text_Type'] = df1_br_types
df2_broadsheets['Closest_Text_Type'] = df2_br_types

In [24]:
df1_broadsheets = df1_broadsheets.drop(['Closest Text Type'],axis=1)
df1_tabloids = df1_tabloids.drop(['Closest Text Type'],axis=1)
df2_broadsheets = df2_broadsheets.drop(['Closest Text Type'],axis=1)
df2_tabloids = df2_tabloids.drop(['Closest Text Type'],axis=1)

## Features (количественные характеристики): 

* Tokens – количество токенов.
* AWL – средняя длина слова.
* TTR – количество тренировочных токенов (400).
* AMP – наличие слов-усилителей.
* ANDC – управление независимой клаузой.
* CAUS – наличие обстоятельств причины по типу «because».
* CC – ???.
* CD – кардинальное число (элементы множества).
* CONC – концессионная обусловленность (наличие слов типа «although», «though» и т.п.)
* COND – слова состояния, условности (наличие слов типа «if», «unless» и т.п.)
* CONJ – слова-союзы.
* DEMO – указательные местоимения, которые не подходят ни к одному из следующих тэгов: DEMP, TOBJ, TSUB, THAC или THVC.
* DEMP – указательное местоимение.
* DT – слово-определитель.
* DWNT – слова, обозначающие незавершённость действия («almost», «hardly», «somewhat», «» и многие другие).
* EMPH – слова с эмоциональным оттенком («really», «just», «for sure» и т.п.)
* EX – слова с указанием существования чего-либо («there»).
* FPP1 – местоимения первого лица («I», «we», «our», «my», «ourselves», «us», «me», «myself»).
* FW – ???.
* GER – герундий (глаголы с «-ing» на конце).
* HDG – показатели неопределённости («more or less», «maybe», «something like» и т.п.)
* IN – подчинительные союзы.
* INPR – местоимения без точного указания на конкретное лицо («everyone», «anyone», «someone», «something» и т.п.)
* JJ – прилагательные.
* NEMD – глаголы обязательства («ought», «should», «must»).
* NN – существительные.
* NOMZ – отглагольные существительные («-tion», «-ness», «-ment» и т.п).
* OSUB – слова по типу «since», «while», «whilst», «whereupon», «whereas», «whereby», «such that», «so that».
* PDT – слова, стоящие перед определением.
* PHC – союз «и».
* PIN – предлоги.
* PIT – местоимение «it».
* PLACE – указатели места («above», «below», «», «» и т.п.).
* POMD – модальные глаголы возможности («can», «may», «might», «could»).
* PRED – предикативное прилагательное.
* PRMD – глаголы с указанием на будущее время («will», «shall», «would»).
* PRP – личное местоимение.
* QUAN – числительное.
* QUPR – числительное местоимение.
* RB – наречие.
* RP – ???.
* SPP2 – местоимения в форме второго лица («you», «your», «yourself», «yourselves», «thy», «thee», «thyself», «thou»).
* SYM – ???.
* SYNE – конструкции «no» + прилагательное.
* THAC – конструкции «that» + прилагательное.
* THVC – конструкции «that» + глагол.
* TIME – времени («afterwards», «again», «earlier», «early» и другие).
* TO – инфинитивы, которые следуют после слова «to».
* TOBJ – конструкции по типу «that» + подлежащее + сказуемое; применяется в отношении объекта предложения.
* TPP3 – местоимения в форме третьего лица («she», «he», «they», «her», «him», «them», «his», «their», «himself», «herself», «themselves»).
* TSUB – конструкции по типу «that» + подлежащее + сказуемое; применяется в отношении субъекта предложения.
* UH – ???.
* VB – глаголы.
* VBD – глаголы в форме прошедшего времени.
* VBG – глаголы с формой Present Continuous с «-ing» на конце.
* VBN – глаголы с формой Past Continuous с «-ing» на конце.
* VPRT – глаголы с формой Present Simple.
* WDT – ???.
* WP – местоимения, начинающиеся на «wh-».
* WPS – местоимения, начинающиеся на «wh-» и имеющие посессивный характер.
* XX0 – отрицания; слова «not» и «n’t».
* [BEMA] – глагольная фраза с «be».
* [BYPA] – фразы в пассивном залоге с «by».
* [PASS] – фразы в пассивном залоге без пациенса.
* [PEAS] – фразы в перфекте («have» + глагол).
* [PIRE] – предикаты сложноподчинённого предложения со словами «who», «whose» или «which»; причём эти слова обозначают субъект из основной части предложения.
* [PRIV] – список глаголов по Quirk et al. (1985: 1181–2): «feel», «recall», «accept», «assume», «prove», «believe», «remember», «calculate» и т.п., в т.ч. их словоформы.
* [PROD] – использования слова «do» в качестве самостоятельного глагола.
* [PUBV] – глаголы по типу «announce», «certify», «say», «write» и т.п.
* [SMP] – глаголы «seem» и «appear».
* [SPAU] – конструкции типа «вспомогательный глагол + минимум одно наречие + базовая форма глагола».
* [SPIN] – конструкции типа «to + минимум одно наречие + базовая форма глагола».
* [SUAV] – список глаголов убеждения по Quirk et al. (1985: 1182–3): «ensure», «insist», «order», «propose», «request», «suggest» и т.п.
* [THATD] – подчинительные клаузы без «that».
* [WHCL] – клаузы с вопросительными словами на «wh-», после которых следует слово, не являющееся вспомогательной частью речи.
* [WHOBJ] – определённые придаточные предложения, имеющие вопросительные слова на «wh-» в качестве объекта.
* [WHSUB] – определённые придаточные предложения, имеющие вопросительные слова на «wh-» в качестве субъекта.
* [WZPAST] – фразы по типу «The solution produced by this process»
* [WZPRES] – фразы по типу «the event causing this decline is…»

In [25]:
dataset = []
feature = []
score = []

In [26]:
df2_broadsheets.columns

Index(['Tokens', 'AWL', 'TTR', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD', 'CONC',
       'COND', 'CONJ', 'DEMO', 'DEMP', 'DT', 'DWNT', 'EMPH', 'EX', 'FPP1',
       'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'NEMD', 'NN', 'NOMZ', 'OSUB',
       'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED', 'PRMD',
       'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC',
       'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG',
       'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[PASS]',
       '[PASTP]', '[PEAS]', '[PIRE]', '[PRIV]', '[PROD]', '[PUBV]', '[SERE]',
       '[SMP]', '[SPAU]', '[SPIN]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]',
       '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]', 'Closest_Text_Type'],
      dtype='object')

### Корреляция по broadsheets (dimensions)

In [27]:
correl = df1_broadsheets.corr()['Closest_Text_Type'][df1_broadsheets.corr()['Closest_Text_Type'] != 1].sort_values(ascending=True)
feature = correl.index
score = correl.values
correlation = pd.DataFrame()
correlation['Параметр'] = feature
correlation['Корреляция'] = score

correlation

,Параметр,Корреляция
0,Dimension1,-0.515000
1,Dimension2,-0.410130
2,Dimension4,-0.173493
3,Dimension6,-0.030158
4,Dimension5,0.015463
5,Dimension3,0.332876


In [28]:
correl = df1_broadsheets.corr()['Closest_Text_Type'][df1_broadsheets.corr()['Closest_Text_Type'] != 1].sort_values(ascending=False).head()
feature = correl.index
score = correl.values
correlation = pd.DataFrame()
correlation['Параметр'] = feature
correlation['Корреляция'] = score

correlation

,Параметр,Корреляция
0,Dimension3,0.332876
1,Dimension5,0.015463
2,Dimension6,-0.030158
3,Dimension4,-0.173493
4,Dimension2,-0.410130


In [29]:
from scipy import stats

print(stats.pearsonr(df1_broadsheets['Dimension1'], df1_broadsheets['Closest_Text_Type']))
print(stats.pearsonr(df1_broadsheets['Dimension2'], df1_broadsheets['Closest_Text_Type']))
print(stats.pearsonr(df1_broadsheets['Dimension4'], df1_broadsheets['Closest_Text_Type']))
print(stats.pearsonr(df1_broadsheets['Dimension6'], df1_broadsheets['Closest_Text_Type']))
print(stats.pearsonr(df1_broadsheets['Dimension5'], df1_broadsheets['Closest_Text_Type']))
print(stats.pearsonr(df1_broadsheets['Dimension3'], df1_broadsheets['Closest_Text_Type']))

PearsonRResult(statistic=-0.514999592795047, pvalue=1.205634754338842e-10)
PearsonRResult(statistic=-0.4101304697638328, pvalue=6.450551314705333e-07)
PearsonRResult(statistic=-0.17349292647152822, pvalue=0.04261350108114917)
PearsonRResult(statistic=-0.03015793066054716, pvalue=0.726462669530383)
PearsonRResult(statistic=0.01546297645947228, pvalue=0.857669264971359)
PearsonRResult(statistic=0.332876018813791, pvalue=7.053622033389623e-05)


### Корреляция по broadsheets (Features)

In [30]:
df2_broadsheets = df2_broadsheets.loc[:,~df2_broadsheets.columns.duplicated()].copy()
df2_tabloids = df2_tabloids.loc[:,~df2_tabloids.columns.duplicated()].copy()

In [31]:
correl = df2_broadsheets.corr()['Closest_Text_Type'][df2_broadsheets.corr()['Closest_Text_Type'] != 1].sort_values(ascending=True).head()
feature = correl.index
feature1 = feature
score = correl.values
correlation = pd.DataFrame()
pearsonr = []
for f in feature:
    pearsonr.append(stats.pearsonr(df2_broadsheets[f], df2_broadsheets['Closest_Text_Type'])[1])

correlation['Feature'] = feature
correlation['Correlation'] = score
correlation['P-value'] = pearsonr

correlation

,Feature,Correlation,P-value
0,FPP1,-0.387537,0.000003
1,TPP3,-0.358187,0.000017
2,VB,-0.345953,0.000035
3,XX0,-0.328693,0.000088
4,PIT,-0.324777,0.000108


In [32]:
correl = df2_broadsheets.corr()['Closest_Text_Type'][df2_broadsheets.corr()['Closest_Text_Type'] != 1].sort_values(ascending=False).head()
feature = correl.index
feature2 = feature
score = correl.values

pearsonr = []
for f in feature:
    pearsonr.append(stats.pearsonr(df2_broadsheets[f], df2_broadsheets['Closest_Text_Type'])[1])

correlation = pd.DataFrame()
correlation['Параметр'] = feature
correlation['Корреляция'] = score
correlation['P-value'] = pearsonr

    
correlation

,Параметр,Корреляция,P-value
0,AWL,0.426358,2.049407e-07
1,PHC,0.408147,7.390863e-07
2,NN,0.348608,2.983818e-05
3,JJ,0.326417,9.909812e-05
4,PIN,0.259640,2.184848e-03


### Корреляция по tabloids (dimensions)

In [33]:
correl = df1_tabloids.corr()['Closest_Text_Type'][df1_tabloids.corr()['Closest_Text_Type'] != 1].sort_values(ascending=False).head()
feature = correl.index
score = correl.values

correlation = pd.DataFrame()
correlation['Dimension'] = feature
correlation['Корреляция'] = score

correlation

,Dimension,Корреляция
0,Dimension3,0.269407
1,Dimension5,0.227036
2,Dimension4,-0.053711
3,Dimension6,-0.114513
4,Dimension1,-0.158765


### Корреляция по tabloids (основные параметры)

In [34]:
correl = df2_tabloids.corr()['Closest_Text_Type'][df2_tabloids.corr()['Closest_Text_Type'] != 1].sort_values(ascending=True).head()
feature = correl.index
feature3 = feature
score = correl.values

pearsonr = []
for f in feature:
    pearsonr.append(stats.pearsonr(df2_tabloids[f], df2_tabloids['Closest_Text_Type'])[1])

correlation = pd.DataFrame()
correlation['Параметр'] = feature
correlation['Корреляция'] = score
correlation['P-value'] = pearsonr

correlation

,Параметр,Корреляция,P-value
0,TPP3,-0.243757,0.004386
1,SYNE,-0.226020,0.008391
2,TIME,-0.223909,0.009037
3,Tokens,-0.202692,0.018389
4,TOBJ,-0.185198,0.031521


In [35]:
correl = df2_tabloids.corr()['Closest_Text_Type'][df2_tabloids.corr()['Closest_Text_Type'] != 1].sort_values(ascending=False).head()
feature = correl.index
feature4 = feature
score = correl.values

pearsonr = []
for f in feature:
    pearsonr.append(stats.pearsonr(df2_tabloids[f], df2_tabloids['Closest_Text_Type'])[1])

correlation = pd.DataFrame()
correlation['Параметр'] = feature
correlation['Корреляция'] = score
correlation['P-value'] = pearsonr

correlation

,Параметр,Корреляция,P-value
0,AWL,0.242714,0.004563
1,JJ,0.238583,0.005325
2,NOMZ,0.184365,0.032306
3,GER,0.179405,0.037339
4,PHC,0.171176,0.047137


In [36]:
features_full = feature1.to_list() + feature2.to_list() + feature3.to_list() + feature4.to_list()
set(features_full)

{'AWL',
 'FPP1',
 'GER',
 'JJ',
 'NN',
 'NOMZ',
 'PHC',
 'PIN',
 'PIT',
 'SYNE',
 'TIME',
 'TOBJ',
 'TPP3',
 'Tokens',
 'VB',
 'XX0'}

In [37]:
df2_tabloids = df2_tabloids[[
                                         'AWL',
                                         'FPP1',
                                         'GER',
                                         'JJ',
                                         'NN',
                                         'NOMZ',
                                         'PHC',
                                         'PIN',
                                         'PIT',
                                         'SYNE',
                                         'TIME',
                                         'TOBJ',
                                         'TPP3',
                                         'Tokens',
                                         'VB',
                                         'XX0',
                                         'Closest_Text_Type']]

df2_broadsheets = df2_broadsheets[[
                                         'AWL',
                                         'FPP1',
                                         'GER',
                                         'JJ',
                                         'NN',
                                         'NOMZ',
                                         'PHC',
                                         'PIN',
                                         'PIT',
                                         'SYNE',
                                         'TIME',
                                         'TOBJ',
                                         'TPP3',
                                         'Tokens',
                                         'VB',
                                         'XX0',
                                         'Closest_Text_Type']]

In [38]:
df1t_train = df1_tabloids[:72]
df2t_train = df2_tabloids[:72]
df1b_train = df1_broadsheets[:73]
df2b_train = df2_broadsheets[:73]

In [39]:
df1t_test = df1_tabloids[72:]
df2t_test = df2_tabloids[72:]
df1b_test = df1_broadsheets[73:]
df2b_test = df2_broadsheets[73:]

In [40]:
df1_tabloids.to_csv('tabloids_dimensions.csv', index=False)

In [41]:
df2_tabloids.to_csv('tabloids_features.csv', index=False)

In [42]:
df1_broadsheets.to_csv('broadsheets_dimensions.csv', index=False)

In [43]:
df2_broadsheets.to_csv('broadsheets_features.csv', index=False)

In [44]:
y_train1_t = df1t_train['Closest_Text_Type']
y_test1_t = df1t_test['Closest_Text_Type']

x_train1_t = df1t_train.drop(['Closest_Text_Type'], axis=1)
x_test1_t = df1t_test.drop(['Closest_Text_Type'], axis=1)

In [45]:
y_train1_b = df1b_train['Closest_Text_Type']
y_test1_b = df1b_test['Closest_Text_Type']

x_train1_b = df1b_train.drop(['Closest_Text_Type'], axis=1)
x_test1_b = df1b_test.drop(['Closest_Text_Type'], axis=1)

In [46]:
y_train2_t = df2t_train['Closest_Text_Type']
y_test2_t = df2t_test['Closest_Text_Type']

x_train2_t = df2t_train.drop(['Closest_Text_Type'], axis=1)
x_test2_t = df2t_test.drop(['Closest_Text_Type'], axis=1)

In [47]:
y_train2_b = df2b_train['Closest_Text_Type']
y_test2_b = df2b_test['Closest_Text_Type']

x_train2_b = df2b_train.drop(['Closest_Text_Type'], axis=1)
x_test2_b = df2b_test.drop(['Closest_Text_Type'], axis=1)

In [48]:
x_train2_b

,AWL,FPP1,GER,JJ,NN,NOMZ,PHC,PIN,PIT,SYNE,TIME,TOBJ,TPP3,Tokens,VB,XX0
0,4.51,0.49,0.24,10.46,23.11,4.38,0.73,11.92,1.22,0.73,0.24,0.00,4.14,411.0,9.98,0.49
1,4.38,0.00,0.00,6.06,32.12,1.82,0.61,17.58,0.00,0.00,1.21,0.00,3.03,165.0,9.70,0.00
2,4.22,4.14,0.00,7.59,22.99,2.76,0.46,6.67,1.38,0.23,1.15,0.46,5.06,435.0,12.64,1.38
3,4.79,0.85,0.43,9.51,30.66,1.07,1.50,11.00,0.96,0.00,0.53,0.11,2.46,936.0,7.69,0.75
4,4.62,2.92,0.00,9.91,31.78,0.29,0.87,9.62,1.75,0.00,0.00,0.00,0.58,343.0,10.79,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,4.41,2.16,0.87,8.23,30.30,0.00,0.00,11.26,0.87,0.00,0.43,0.00,3.46,231.0,11.26,0.43
69,4.31,2.54,0.60,7.17,23.77,1.79,0.75,8.67,1.20,0.00,0.30,0.00,3.29,669.0,10.91,1.49
70,4.37,2.44,0.49,9.12,23.62,2.61,0.81,10.26,0.81,0.49,0.49,0.00,3.42,614.0,13.36,0.81
71,4.81,0.97,0.65,10.03,33.01,3.24,0.97,12.30,0.65,0.00,0.32,0.65,1.94,309.0,7.12,0.32


# Нейросеть

### Нейросеть: Tabloids (Dimensions)

In [49]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from sklearn.metrics import accuracy_score

algorithms = []
datasets = []
features = []
accuracies = []

In [50]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(x_train1_t)

(trainX, testX, trainY, testY) = train_test_split(X_scale,
    y_train1_t, test_size=0.5, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

model = Sequential()
model.add(Dense(1024, input_shape=(6,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=100, batch_size=32)

predictions = model.predict(testX, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = testY.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Tabloids (обучающая выборка)')
features.append('Dimensions')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
2/2 [==============================] - 1s 142ms/step - loss: 1.1216 - accuracy: 0.3056 - val_loss: 1.9932 - val_accuracy: 0.1944
Epoch 2/100
2/2 [==============================] - 0s 26ms/step - loss: 2.3780 - accuracy: 0.3056 - val_loss: 1.2150 - val_accuracy: 0.6667
Epoch 3/100
2/2 [==============================] - 0s 26ms/step - loss: 1.8510 - accuracy: 0.4444 - val_loss: 0.8657 - val_accuracy: 0.6667
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 1.1138 - accuracy: 0.4722 - val_loss: 2.3886 - val_accuracy: 0.1389
Epoch 5/100
2/2 [==============================] - 0s 27ms/step - loss: 2.0376 - accuracy: 0.2500 - val_loss: 1.4771 - val_accuracy: 0.1389
Epoch 6/100
2/2 [==============================] - 0s 26ms/step - loss: 1.2351 - accuracy: 0.3056 - val_loss: 0.9993 - val_accuracy: 0.6667
Epoch 7/100
2/2 [==============================] - 0s 27ms/step - loss: 1.3263 - accuracy: 0.4444 - val_loss: 1.0775 - val_accuracy: 0.6667
Epoch 8/100
2/2 [==

2/2 [==============================] - 0s 28ms/step - loss: 1.0477 - accuracy: 0.6944 - val_loss: 0.9077 - val_accuracy: 0.6667
Epoch 60/100
2/2 [==============================] - 0s 27ms/step - loss: 1.0435 - accuracy: 0.4444 - val_loss: 0.8562 - val_accuracy: 0.6667
Epoch 61/100
2/2 [==============================] - 0s 28ms/step - loss: 1.1237 - accuracy: 0.4444 - val_loss: 0.8637 - val_accuracy: 0.6667
Epoch 62/100
2/2 [==============================] - 0s 27ms/step - loss: 1.2026 - accuracy: 0.4444 - val_loss: 0.8611 - val_accuracy: 0.6667
Epoch 63/100
2/2 [==============================] - 0s 25ms/step - loss: 1.1666 - accuracy: 0.4444 - val_loss: 0.8803 - val_accuracy: 0.6667
Epoch 64/100
2/2 [==============================] - 0s 26ms/step - loss: 1.0617 - accuracy: 0.4444 - val_loss: 0.9895 - val_accuracy: 0.6667
Epoch 65/100
2/2 [==============================] - 0s 28ms/step - loss: 1.0503 - accuracy: 0.3889 - val_loss: 1.1479 - val_accuracy: 0.1944
Epoch 66/100
2/2 [========

### Нейросеть: Tabloids (основные параметры)

In [51]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(x_train2_t)

(trainX, testX, trainY, testY) = train_test_split(X_scale,
    y_train2_t, test_size=0.5, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

model = Sequential()
model.add(Dense(1024, input_shape=(16,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=100, batch_size=32)

predictions = model.predict(testX, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = testY.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Tabloids (обучающая выборка)')
features.append('Features')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
2/2 [==============================] - 1s 117ms/step - loss: 1.3761 - accuracy: 0.2778 - val_loss: 1.0736 - val_accuracy: 0.6667
Epoch 2/100
2/2 [==============================] - 0s 27ms/step - loss: 1.6016 - accuracy: 0.5000 - val_loss: 1.9946 - val_accuracy: 0.1944
Epoch 3/100
2/2 [==============================] - 0s 27ms/step - loss: 1.7427 - accuracy: 0.3056 - val_loss: 1.3516 - val_accuracy: 0.1389
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 1.1938 - accuracy: 0.2500 - val_loss: 1.0131 - val_accuracy: 0.6667
Epoch 5/100
2/2 [==============================] - 0s 28ms/step - loss: 1.5234 - accuracy: 0.4444 - val_loss: 1.0601 - val_accuracy: 0.6667
Epoch 6/100
2/2 [==============================] - 0s 28ms/step - loss: 1.6290 - accuracy: 0.4444 - val_loss: 0.9263 - val_accuracy: 0.6667
Epoch 7/100
2/2 [==============================] - 0s 28ms/step - loss: 1.1796 - accuracy: 0.3889 - val_loss: 1.6106 - val_accuracy: 0.1944
Epoch 8/100
2/2 [==

2/2 [==============================] - 0s 27ms/step - loss: 0.9448 - accuracy: 0.4444 - val_loss: 0.8192 - val_accuracy: 0.6667
Epoch 60/100
2/2 [==============================] - 0s 27ms/step - loss: 0.9478 - accuracy: 0.4444 - val_loss: 0.8357 - val_accuracy: 0.6667
Epoch 61/100
2/2 [==============================] - 0s 27ms/step - loss: 0.9258 - accuracy: 0.4444 - val_loss: 0.9048 - val_accuracy: 0.6944
Epoch 62/100
2/2 [==============================] - 0s 27ms/step - loss: 0.9189 - accuracy: 0.5556 - val_loss: 1.0101 - val_accuracy: 0.2222
Epoch 63/100
2/2 [==============================] - 0s 28ms/step - loss: 0.9702 - accuracy: 0.3611 - val_loss: 1.0756 - val_accuracy: 0.1389
Epoch 64/100
2/2 [==============================] - 0s 27ms/step - loss: 1.0085 - accuracy: 0.2500 - val_loss: 0.9514 - val_accuracy: 0.5556
Epoch 65/100
2/2 [==============================] - 0s 26ms/step - loss: 0.9216 - accuracy: 0.5556 - val_loss: 0.8393 - val_accuracy: 0.6944
Epoch 66/100
2/2 [========

In [52]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train1_t)
y_test = lb.transform(y_test1_t)

model = Sequential()
model.add(Dense(1024, input_shape=(6,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(x_train1_t, y_train, validation_data=(x_test1_t, y_test),
    epochs=100, batch_size=32)

predictions = model.predict(x_test1_t, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = y_test.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Tabloids (тестовая выборка)')
features.append('Dimensions')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
3/3 [==============================] - 1s 66ms/step - loss: 1.5877 - accuracy: 0.5417 - val_loss: 1.1395 - val_accuracy: 0.1746
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 1.3083 - accuracy: 0.2639 - val_loss: 0.7974 - val_accuracy: 0.7937
Epoch 3/100
3/3 [==============================] - 0s 17ms/step - loss: 0.9522 - accuracy: 0.6389 - val_loss: 1.1613 - val_accuracy: 0.5714
Epoch 4/100
3/3 [==============================] - 0s 16ms/step - loss: 1.1278 - accuracy: 0.5556 - val_loss: 0.6398 - val_accuracy: 0.6984
Epoch 5/100
3/3 [==============================] - 0s 16ms/step - loss: 0.6466 - accuracy: 0.8194 - val_loss: 0.7235 - val_accuracy: 0.5397
Epoch 6/100
3/3 [==============================] - 0s 16ms/step - loss: 0.7412 - accuracy: 0.5694 - val_loss: 0.5379 - val_accuracy: 0.9683
Epoch 7/100
3/3 [==============================] - 0s 18ms/step - loss: 0.5699 - accuracy: 0.8750 - val_loss: 0.4931 - val_accuracy: 0.7937
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 17ms/step - loss: 0.0788 - accuracy: 1.0000 - val_loss: 0.1362 - val_accuracy: 0.9683
Epoch 60/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0783 - accuracy: 1.0000 - val_loss: 0.1519 - val_accuracy: 0.9206
Epoch 61/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0839 - accuracy: 1.0000 - val_loss: 0.1314 - val_accuracy: 0.9524
Epoch 62/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0734 - accuracy: 1.0000 - val_loss: 0.1194 - val_accuracy: 0.9683
Epoch 63/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0710 - accuracy: 1.0000 - val_loss: 0.1162 - val_accuracy: 0.9524
Epoch 64/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0678 - accuracy: 1.0000 - val_loss: 0.1174 - val_accuracy: 0.9683
Epoch 65/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0668 - accuracy: 1.0000 - val_loss: 0.1199 - val_accuracy: 0.9683
Epoch 66/100
3/3 [========

In [53]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train2_t)
y_test = lb.transform(y_test2_t)

model = Sequential()
model.add(Dense(1024, input_shape=(16,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(x_train2_t, y_train, validation_data=(x_test2_t, y_test),
    epochs=100, batch_size=32)

predictions = model.predict(x_test2_t, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = y_test.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Tabloids (тестовая выборка)')
features.append('Features')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
3/3 [==============================] - 1s 69ms/step - loss: 1.1895 - accuracy: 0.4167 - val_loss: 1.4878 - val_accuracy: 0.5714
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 1.5762 - accuracy: 0.5556 - val_loss: 1.0588 - val_accuracy: 0.5714
Epoch 3/100
3/3 [==============================] - 0s 17ms/step - loss: 1.0349 - accuracy: 0.5139 - val_loss: 1.2246 - val_accuracy: 0.1746
Epoch 4/100
3/3 [==============================] - 0s 17ms/step - loss: 1.1581 - accuracy: 0.2917 - val_loss: 1.0180 - val_accuracy: 0.5714
Epoch 5/100
3/3 [==============================] - 0s 18ms/step - loss: 1.1113 - accuracy: 0.5556 - val_loss: 1.0644 - val_accuracy: 0.5714
Epoch 6/100
3/3 [==============================] - 0s 19ms/step - loss: 1.0455 - accuracy: 0.5694 - val_loss: 1.0429 - val_accuracy: 0.5397
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 1.1233 - accuracy: 0.3611 - val_loss: 1.0530 - val_accuracy: 0.4603
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 17ms/step - loss: 0.4502 - accuracy: 0.7917 - val_loss: 0.5909 - val_accuracy: 0.7619
Epoch 60/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4800 - accuracy: 0.7639 - val_loss: 0.6107 - val_accuracy: 0.7143
Epoch 61/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4300 - accuracy: 0.8056 - val_loss: 0.6722 - val_accuracy: 0.7143
Epoch 62/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4250 - accuracy: 0.7778 - val_loss: 0.6590 - val_accuracy: 0.7460
Epoch 63/100
3/3 [==============================] - 0s 18ms/step - loss: 0.4818 - accuracy: 0.7778 - val_loss: 0.6757 - val_accuracy: 0.6825
Epoch 64/100
3/3 [==============================] - 0s 16ms/step - loss: 0.4589 - accuracy: 0.7778 - val_loss: 0.6007 - val_accuracy: 0.7302
Epoch 65/100
3/3 [==============================] - 0s 16ms/step - loss: 0.4088 - accuracy: 0.8056 - val_loss: 0.6487 - val_accuracy: 0.7619
Epoch 66/100
3/3 [========

### Нейросеть: Broadsheets (Dimensions)

In [54]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(x_train1_b)

(trainX, testX, trainY, testY) = train_test_split(X_scale,
    y_train1_b, test_size=0.5, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

model = Sequential()
model.add(Dense(1024, input_shape=(6,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=100, batch_size=32)

predictions = model.predict(testX, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = testY.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Broadsheets (обучающая выборка)')
features.append('Dimensions')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
2/2 [==============================] - 1s 125ms/step - loss: 1.2747 - accuracy: 0.3056 - val_loss: 1.1310 - val_accuracy: 0.5405
Epoch 2/100
2/2 [==============================] - 0s 27ms/step - loss: 1.0221 - accuracy: 0.5833 - val_loss: 1.0758 - val_accuracy: 0.4054
Epoch 3/100
2/2 [==============================] - 0s 26ms/step - loss: 1.1209 - accuracy: 0.3611 - val_loss: 1.1450 - val_accuracy: 0.5405
Epoch 4/100
2/2 [==============================] - 0s 27ms/step - loss: 1.0678 - accuracy: 0.6389 - val_loss: 1.3316 - val_accuracy: 0.5405
Epoch 5/100
2/2 [==============================] - 0s 28ms/step - loss: 1.1799 - accuracy: 0.6389 - val_loss: 1.0408 - val_accuracy: 0.5405
Epoch 6/100
2/2 [==============================] - 0s 27ms/step - loss: 1.0651 - accuracy: 0.5278 - val_loss: 1.0243 - val_accuracy: 0.4054
Epoch 7/100
2/2 [==============================] - 0s 27ms/step - loss: 1.0600 - accuracy: 0.3056 - val_loss: 1.0127 - val_accuracy: 0.5405
Epoch 8/100
2/2 [==

2/2 [==============================] - 0s 29ms/step - loss: 0.8014 - accuracy: 0.6389 - val_loss: 0.8140 - val_accuracy: 0.5405
Epoch 60/100
2/2 [==============================] - 0s 27ms/step - loss: 0.8079 - accuracy: 0.6389 - val_loss: 0.8266 - val_accuracy: 0.7838
Epoch 61/100
2/2 [==============================] - 0s 27ms/step - loss: 0.8550 - accuracy: 0.6944 - val_loss: 0.8189 - val_accuracy: 0.6757
Epoch 62/100
2/2 [==============================] - 0s 28ms/step - loss: 0.8368 - accuracy: 0.7222 - val_loss: 0.8095 - val_accuracy: 0.5405
Epoch 63/100
2/2 [==============================] - 0s 27ms/step - loss: 0.8104 - accuracy: 0.6389 - val_loss: 0.8110 - val_accuracy: 0.5405
Epoch 64/100
2/2 [==============================] - 0s 28ms/step - loss: 0.7910 - accuracy: 0.6389 - val_loss: 0.8152 - val_accuracy: 0.5405
Epoch 65/100
2/2 [==============================] - 0s 26ms/step - loss: 0.7863 - accuracy: 0.6389 - val_loss: 0.8210 - val_accuracy: 0.5405
Epoch 66/100
2/2 [========

In [55]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train1_b)
y_test = lb.transform(y_test1_b)

model = Sequential()
model.add(Dense(1024, input_shape=(6,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(x_train1_b, y_train, validation_data=(x_test1_b, y_test),
    epochs=100, batch_size=32)

predictions = model.predict(x_test1_b, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = y_test.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Broadsheets (тестовая выборка)')
features.append('Dimensions')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
3/3 [==============================] - 1s 65ms/step - loss: 1.0571 - accuracy: 0.5890 - val_loss: 1.3272 - val_accuracy: 0.5469
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 1.0584 - accuracy: 0.6164 - val_loss: 1.0497 - val_accuracy: 0.8438
Epoch 3/100
3/3 [==============================] - 0s 16ms/step - loss: 0.8296 - accuracy: 0.7808 - val_loss: 0.9017 - val_accuracy: 0.5938
Epoch 4/100
3/3 [==============================] - 0s 19ms/step - loss: 0.6803 - accuracy: 0.7123 - val_loss: 0.8056 - val_accuracy: 0.6562
Epoch 5/100
3/3 [==============================] - 0s 17ms/step - loss: 0.6430 - accuracy: 0.8493 - val_loss: 0.6878 - val_accuracy: 0.6562
Epoch 6/100
3/3 [==============================] - 0s 16ms/step - loss: 0.5452 - accuracy: 0.7671 - val_loss: 0.6839 - val_accuracy: 0.6562
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 0.5180 - accuracy: 0.7534 - val_loss: 0.6376 - val_accuracy: 0.8438
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 16ms/step - loss: 0.0511 - accuracy: 1.0000 - val_loss: 0.2783 - val_accuracy: 0.9062
Epoch 60/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0452 - accuracy: 1.0000 - val_loss: 0.2790 - val_accuracy: 0.9062
Epoch 61/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0426 - accuracy: 1.0000 - val_loss: 0.2790 - val_accuracy: 0.9062
Epoch 62/100
3/3 [==============================] - 0s 15ms/step - loss: 0.0432 - accuracy: 1.0000 - val_loss: 0.2836 - val_accuracy: 0.8906
Epoch 63/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0430 - accuracy: 1.0000 - val_loss: 0.2806 - val_accuracy: 0.8906
Epoch 64/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0386 - accuracy: 1.0000 - val_loss: 0.2835 - val_accuracy: 0.8906
Epoch 65/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0423 - accuracy: 1.0000 - val_loss: 0.2872 - val_accuracy: 0.8906
Epoch 66/100
3/3 [========

### Нейросеть: Broadsheets (основные параметры)

In [56]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(x_train2_b)

(trainX, testX, trainY, testY) = train_test_split(X_scale,
    y_train2_b, test_size=0.5, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

model = Sequential()
model.add(Dense(1024, input_shape=(16,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=100, batch_size=32)

predictions = model.predict(testX, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = testY.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Broadsheets (обучающая выборка)')
features.append('Features')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
2/2 [==============================] - 1s 126ms/step - loss: 1.3670 - accuracy: 0.3056 - val_loss: 1.5043 - val_accuracy: 0.5405
Epoch 2/100
2/2 [==============================] - 0s 33ms/step - loss: 1.3055 - accuracy: 0.6389 - val_loss: 0.9079 - val_accuracy: 0.5405
Epoch 3/100
2/2 [==============================] - 0s 31ms/step - loss: 0.8903 - accuracy: 0.6389 - val_loss: 0.9104 - val_accuracy: 0.5405
Epoch 4/100
2/2 [==============================] - 0s 30ms/step - loss: 0.8898 - accuracy: 0.6389 - val_loss: 1.0038 - val_accuracy: 0.5405
Epoch 5/100
2/2 [==============================] - 0s 32ms/step - loss: 0.9118 - accuracy: 0.6389 - val_loss: 0.9957 - val_accuracy: 0.5405
Epoch 6/100
2/2 [==============================] - 0s 31ms/step - loss: 0.8948 - accuracy: 0.6389 - val_loss: 0.8737 - val_accuracy: 0.5405
Epoch 7/100
2/2 [==============================] - 0s 31ms/step - loss: 0.8110 - accuracy: 0.6389 - val_loss: 0.8785 - val_accuracy: 0.7297
Epoch 8/100
2/2 [==

2/2 [==============================] - 0s 28ms/step - loss: 0.7869 - accuracy: 0.6389 - val_loss: 0.8547 - val_accuracy: 0.5405
Epoch 60/100
2/2 [==============================] - 0s 28ms/step - loss: 0.7434 - accuracy: 0.6389 - val_loss: 0.7319 - val_accuracy: 0.6216
Epoch 61/100
2/2 [==============================] - 0s 31ms/step - loss: 0.6911 - accuracy: 0.6944 - val_loss: 0.7799 - val_accuracy: 0.5676
Epoch 62/100
2/2 [==============================] - 0s 31ms/step - loss: 0.8187 - accuracy: 0.4167 - val_loss: 0.7286 - val_accuracy: 0.7568
Epoch 63/100
2/2 [==============================] - 0s 26ms/step - loss: 0.7542 - accuracy: 0.7500 - val_loss: 0.7224 - val_accuracy: 0.5676
Epoch 64/100
2/2 [==============================] - 0s 26ms/step - loss: 0.6714 - accuracy: 0.6667 - val_loss: 0.7111 - val_accuracy: 0.5946
Epoch 65/100
2/2 [==============================] - 0s 29ms/step - loss: 0.6499 - accuracy: 0.6944 - val_loss: 0.7059 - val_accuracy: 0.7838
Epoch 66/100
2/2 [========

In [57]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train2_b)
y_test = lb.transform(y_test2_b)

model = Sequential()
model.add(Dense(1024, input_shape=(16,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(x_train2_b, y_train, validation_data=(x_test2_b, y_test),
    epochs=100, batch_size=32)

predictions = model.predict(x_test2_b, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = y_test.argmax(axis=1)
print(accuracy_score(testY_arg, predictions2))

algorithms.append('Нейросеть')
datasets.append('Broadsheets (тестовая выборка)')
features.append('Features')
accuracies.append(accuracy_score(testY_arg, predictions2))

Epoch 1/100
3/3 [==============================] - 1s 68ms/step - loss: 1.1852 - accuracy: 0.5890 - val_loss: 1.2867 - val_accuracy: 0.5469
Epoch 2/100
3/3 [==============================] - 0s 18ms/step - loss: 1.1274 - accuracy: 0.5342 - val_loss: 1.2307 - val_accuracy: 0.4531
Epoch 3/100
3/3 [==============================] - 0s 17ms/step - loss: 1.0455 - accuracy: 0.5890 - val_loss: 1.1443 - val_accuracy: 0.5469
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 0.9513 - accuracy: 0.5890 - val_loss: 1.0937 - val_accuracy: 0.3906
Epoch 5/100
3/3 [==============================] - 0s 18ms/step - loss: 1.0571 - accuracy: 0.4795 - val_loss: 1.0391 - val_accuracy: 0.5469
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 0.9538 - accuracy: 0.5890 - val_loss: 1.0841 - val_accuracy: 0.5469
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 0.9588 - accuracy: 0.5616 - val_loss: 1.1071 - val_accuracy: 0.5000
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 16ms/step - loss: 0.5085 - accuracy: 0.7808 - val_loss: 0.6636 - val_accuracy: 0.7656
Epoch 60/100
3/3 [==============================] - 0s 17ms/step - loss: 0.7073 - accuracy: 0.6986 - val_loss: 0.6960 - val_accuracy: 0.7344
Epoch 61/100
3/3 [==============================] - 0s 16ms/step - loss: 0.5376 - accuracy: 0.7671 - val_loss: 0.6660 - val_accuracy: 0.7031
Epoch 62/100
3/3 [==============================] - 0s 16ms/step - loss: 0.5184 - accuracy: 0.7534 - val_loss: 0.7212 - val_accuracy: 0.6719
Epoch 63/100
3/3 [==============================] - 0s 16ms/step - loss: 0.5025 - accuracy: 0.7671 - val_loss: 0.7407 - val_accuracy: 0.7031
Epoch 64/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4997 - accuracy: 0.7671 - val_loss: 0.6554 - val_accuracy: 0.7188
Epoch 65/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4653 - accuracy: 0.8219 - val_loss: 0.6869 - val_accuracy: 0.7031
Epoch 66/100
3/3 [========

# Пайплайн

### Пайплайн: Tabloids (Dimensions)

In [58]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)

print(pipe.fit(x_train1_t, y_train1_t).score(x_test1_t, y_test1_t))

algorithms.append('Пайплайн')
datasets.append('Tabloids')
features.append('Dimensions')
accuracies.append(pipe.fit(x_train1_t, y_train1_t).score(x_test1_t, y_test1_t))

0.9365079365079365


### Пайплайн: Tabloids (Основные параметры)

In [59]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)

print(pipe.fit(x_train2_t, y_train2_t).score(x_test2_t, y_test2_t))

algorithms.append('Пайплайн')
datasets.append('Tabloids')
features.append('Features')
accuracies.append(pipe.fit(x_train2_t, y_train2_t).score(x_test2_t, y_test2_t))

0.7777777777777778


In [60]:
train_test_split(x_train2_t, test_size=0.5, random_state=42)

[     AWL  FPP1   GER     JJ     NN  NOMZ   PHC    PIN   PIT  SYNE  TIME  TOBJ  \
 64  4.61  0.00  0.00   7.00  28.01  1.68  1.12  11.20  1.12  0.00  1.40  0.00   
 3   4.16  2.22  0.00  10.16  31.11  0.63  0.00  12.06  0.00  0.00  0.32  0.00   
 17  4.51  1.87  0.75   7.12  23.97  1.87  0.19  10.11  2.25  0.37  0.75  0.19   
 38  4.53  0.00  2.51   6.53  36.18  2.01  1.51  13.57  0.00  0.50  0.00  0.00   
 8   4.87  0.59  1.07   9.24  26.90  3.79  0.59  10.66  0.95  0.00  1.18  0.00   
 57  4.43  2.37  0.64   6.03  26.38  1.45  0.97  10.74  0.97  0.36  1.77  0.08   
 6   4.89  0.00  0.64   9.13  38.85  2.12  1.06  12.53  0.21  0.00  0.21  0.00   
 36  4.45  1.07  0.64   4.51  32.62  1.29  0.64  11.16  1.93  0.00  0.64  0.43   
 66  4.23  2.66  0.42   5.88  21.71  3.78  0.00   9.52  2.10  0.00  1.26  0.00   
 53  4.42  1.18  0.77   6.59  28.68  1.47  1.12  11.65  1.18  0.25  0.73  0.00   
 70  4.09  2.53  0.42   4.21  27.37  1.26  1.05   8.21  1.26  0.21  0.63  0.21   
 15  4.30  3.10 

### Пайплайн: Broadsheets (Dimensions)

In [61]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)
print(pipe.fit(x_train1_b, y_train1_b).score(x_test1_b, y_test1_b))

algorithms.append('Пайплайн')
datasets.append('Broadsheets')
features.append('Dimensions')
accuracies.append(pipe.fit(x_train1_b, y_train1_b).score(x_test1_b, y_test1_b))

0.84375


### Пайплайн: Broadsheets (основные параметры)

In [62]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)
print(pipe.fit(x_train2_b, y_train2_b).score(x_test2_b, y_test2_b))

algorithms.append('Пайплайн')
datasets.append('Broadsheets')
features.append('Features')
accuracies.append(pipe.fit(x_train2_b, y_train2_b).score(x_test2_b, y_test2_b))

0.734375


### Таблица сравнения

In [63]:
comparison = pd.DataFrame()
comparison['Алгоритм'] = algorithms
comparison['Датасет'] = datasets
comparison['Параметры'] = features
comparison['Точность'] = accuracies

In [64]:
comparison

,Алгоритм,Датасет,Параметры,Точность
0,Нейросеть,Tabloids (обучающая выборка),Dimensions,0.194444
1,Нейросеть,Tabloids (обучающая выборка),Features,0.638889
2,Нейросеть,Tabloids (тестовая выборка),Dimensions,0.952381
3,Нейросеть,Tabloids (тестовая выборка),Features,0.746032
4,Нейросеть,Broadsheets (обучающая выборка),Dimensions,0.540541
5,Нейросеть,Broadsheets (тестовая выборка),Dimensions,0.906250
6,Нейросеть,Broadsheets (обучающая выборка),Features,0.756757
7,Нейросеть,Broadsheets (тестовая выборка),Features,0.734375
8,Пайплайн,Tabloids,Dimensions,0.936508
9,Пайплайн,Tabloids,Features,0.777778


### Можно сделать следующие выводы:
* Нейросеть даёт высокую точность как на Tabloids, так и на Broadsheets, и в целом справляется с этой задачей лучше, чем пайплайн.
* Пайплайн лучше обрабатывает датасеты Tabloids, чем Broadsheets, и при этом даёт более высокий результат на параметрах Dimensions. То же самое можно сказать и о нейросети.
* При обработке датасетов через нейросеть можно проследить одну особенность: обработка Tabloids через Dimensions на обучающей выборке даёт меньшую точность, чем при обработке тестовой выборки, однако при обработке тех же данных через Features даёт совершенно противоположную картину: точность на обучающей выборке получилась выше, чем на тестовой. Это можно объяснить тем, что в одной выборке могут быть типы текстов по Байберу, которых нет в другой. Также есть вероятность, что большое количество документов и характеристик по ним может значительно повлиять на качество работы модели, если прослеживается преобладание какого-то одного типа текстов, установленных Байбером.